### Starting with the analysis

Calling global packages

In [1]:
import sys, os

from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env')
load_dotenv()

import pickle, json
import time
import processingData
import collections, itertools, copy, operator

import nltk
import pandas
import gensim
import sklearn
import re
import string
import math
import scipy
import numpy

in the other hello\nworld


Using TensorFlow backend.


# IN processingData.py

### Packages

In [2]:
import os
import re
import nltk
import json
print("in the other",os.getenv("TEST_MULTILINE_VAR"))
nltk.data.path.append(os.getenv("NLTKDATADIR"))
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import copy
import math



in the other hello\nworld


### Functions for first Data Preparation adding POS

In [3]:
############################
## allrecordsPreparation 1: original
############################

def allrecordsPreparation1(allrecords):
    '''
    description: tokenization and POS tagging
    input: dict of allrecords texts and data from different sources
    treatment: separating only those with posts in the forum and tokenizing the posts
    output:
    1) list of lists, each with:
    -- id
    -- username
    -- link of the post
    -- tokenized text
    -- POS tagged text
    2) list of post ids
    '''
    print('in allrecordsPreparation (len(allrecords))::',len(allrecords))
    #global var to this scope
    #forum_tonltk = []
    tktexts = []
    soup_forum = None
    forum_ids = []
    all_posedsts = []
    count = 0
    for u in allrecords:
        #block arguments
        record = u["data"]
        forumpost = record['forum']['foundjob_msg']['text']
        if forumpost == '':
            continue
        forumpostID = record['forum']['foundjob_msg']['id']
        forumpostLINK = record['forum']['foundjob_msg']['link']
        soup_forumpost = BeautifulSoup(forumpost)
        soup_forumpostTEXT = soup_forumpost.find('body').get_text()
        tksoup_forumpostTEXT = nltk.word_tokenize(soup_forumpostTEXT)

        #forum_tonltk.append(soup_forumpostTEXT)

        #listoftexts_forum.append(('f_'+forumpostID,
        #                          [w.lower() for w in tksoup_forumpostTEXT],
        #                          forumpostLINK,
        #                          u))

        modtext = []
        
        ## Get an approximation of sentences
        for w in tksoup_forumpostTEXT:
            w = w.lower()
            rws = []
            if len(w) > 1 and len({'.','-',':'}.intersection(w)) >= 1:
                #print(w)
                for punc in {'.','-',':'}.intersection(w):
                    rws = w.replace(punc, ' '+punc+' ').split()
                #print(rws)
            if len(rws) == 0:
                modtext.append(w)
            else:
                for w in rws:
                    modtext.append(w)

        ## 
        #[w.lower() for w in nltk.word_tokenize(soup_forum.find('body').get_text())]
        all_posedsts.append((
                            'f_'+forumpostID,
                            u["user"],
                            forumpostLINK,
                            modtext,
                            nltk.pos_tag(modtext)
                            ))
        #forum_ids.append(forumpostID)

        count += 1

    print("number of treated posts (len(count)) ::", count)
    #return all_posedsts, forum_ids
    return all_posedsts


############################
## allrecordsPreparation 2: revision of 1, which now includes cleaning and lemmatization plus 
## a single dataset for treated records
############################

def allrecordsPreparation2(allrecords):
    '''
    description: tokenization and POS tagging
    input: dict of allrecords texts and data from different sources
    treatment: separating only those with posts in the forum and tokenizing the posts
    output:
    1) list of lists, each with:
    -- id
    -- username
    -- link of the post
    -- tokenized text
    -- POS tagged text
    2) list of post ids
    '''
    print('in allrecordsPreparation (len(allrecords))::',len(allrecords))
    #global var to this scope
    #forum_tonltk = []
    tktexts = []
    soup_forum = None
    forum_ids = []
    all_lemmpostxts = []
    count = 0
    
       
    def HTMLtoText(u):
        #block arguments
        record = u["data"]
        forumpost = record['forum']['foundjob_msg']['text']
        if forumpost == '':
            return False
        forumpostID = record['forum']['foundjob_msg']['id']
        forumpostLINK = record['forum']['foundjob_msg']['link']
        soup_forumpost = BeautifulSoup(forumpost)
        soup_forumpostTEXT = soup_forumpost.find('body').get_text()
        tksoup_forumpostTEXT = nltk.word_tokenize(soup_forumpostTEXT)
        return forumpostID, forumpostLINK, tksoup_forumpostTEXT
    
    def approxsts(text):
        modtext = []
        for w in tksoup_forumpostTEXT:
            w = w.lower()
            rws = []
            if len(w) > 1 and len({'.','-',':'}.intersection(w)) >= 1:
                #print(w)
                for punc in {'.','-',':'}.intersection(w):
                    rws = w.replace(punc, ' '+punc+' ').split()
                #print(rws)
            if len(rws) == 0:
                modtext.append(w)
            else:
                for w in rws:
                    modtext.append(w)
        return modtext
    
    def lemmatizationofpos(postxt, lemmws):
        lemmposws = []
        for posw in postxt:
            w = posw[0]
            pos = posw[1]
            if nltk.corpus.wordnet.synsets(w):
                try:
                    n = ''
                    if nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms()[0].name().split('.')[0] == 'entity':
                        n = w
                    else:
                        n = nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms()[0].name().split('.')[0]
                    if pos[0] == 'V':
                        n = nltk.stem.wordnet.WordNetLemmatizer().lemmatize(w,'v')
                    if pos == 'NNS' or pos == 'NN$':
                        n = nltk.stem.wordnet.WordNetLemmatizer().lemmatize(w)
                    #print(w, nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms(), n)
                    lemmposws.append((w,True,n,pos))
                    lemmws.append(n)
                except KeyError: #in some cases the POS tag is not recognised by wordnet synset
                    print("pos KeyErrors", w,pos)
                    lemmposws.append((w,False,w,pos))
                    lemmws.append(w)
                except IndexError:
                    print("IndexErrors (not found)", w,pos) #in some cases (w,pos) pair was not found at some point of the synsets root hyernyms
                    #print(nltk.corpus.wordnet.synsets(w,pos[0].lower()))
                    lemmposws.append((w,False,w,pos))
                    lemmws.append(w)
            else:
                #print(w, [])
                lemmposws.append((w,False,w,pos))
                lemmws.append(w)
        return lemmposws
      
    
    lemmws = []
    lemmposrecs = []
    
    for u in allrecords:
        ## Getting the data as a text from HTML format (raw dataset)
        userdata = HTMLtoText(u)
        if userdata != False:
            forumpostID, forumpostLINK, tksoup_forumpostTEXT = userdata
            
            ## Get an approximation of sentences
            modtxt = approxsts(tksoup_forumpostTEXT)
            
            ## POS
            postxt = nltk.pos_tag(modtxt)
            
            ## Lemm text
            lemmpostxt = lemmatizationofpos(postxt,lemmws)
            
            ## Adding data to the new created dataset
            lemmposrecs.append((
                            'f_'+forumpostID,
                            u["user"],
                            forumpostLINK,
                            lemmpostxt
                            ))

    print("number of treated posts (len(count)) ::", count)
    #return all_posedsts, forum_ids
    return lemmposrecs, nltk.FreqDist(lemmws)

### Functions for Lemmatized Dataset

In [4]:
############################
## allrecordsLemmatization1: original
############################

def allrecordsLemmatization1(all_posedsts):
    '''
    description: lemmatization
    input: list of tokenized texts (sentences)
    output:
        1) same list of texts but each with the lematized words when found
        2) freqDist of the words after trying lematization
    '''
    all_lemmposedsts = []
    lemmposedwords = []

    print("-"*15+"\nFails to lematize will be printed below\n"+"-"*15)
    for posedpost in all_posedsts:
        lemmposedws = []
        merged_posedsts = posedpost[4]
                
        for posw in  merged_posedsts:
            w = posw[0]
            pos = posw[1]
            if nltk.corpus.wordnet.synsets(w):
                try:
                    n = ''
                    if nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms()[0].name().split('.')[0] == 'entity':
                        n = w
                    else:
                        n = nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms()[0].name().split('.')[0]
                    if pos[0] == 'V':
                        n = nltk.stem.wordnet.WordNetLemmatizer().lemmatize(w,'v')
                    if pos == 'NNS' or pos == 'NN$':
                        n = nltk.stem.wordnet.WordNetLemmatizer().lemmatize(w)
                    #print(w, nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms(), n)
                    lemmposedws.append(n)
                    lemmposedwords.append(n)
                except KeyError: #in some cases the POS tag is not recognised by wordnet synset
                    print("pos KeyErrors", w,pos)
                    lemmposedws.append(w)
                    lemmposedwords.append(w)
                except IndexError:
                    print("IndexErrors (not found)", w,pos) #in some cases (w,pos) pair was not found at some point of the synsets root hyernyms
                    #print(nltk.corpus.wordnet.synsets(w,pos[0].lower()))
                    lemmposedws.append(w)
                    lemmposedwords.append(w)                    
            else:
                #print(w, [])
                lemmposedws.append(w)
                lemmposedwords.append(w)
        
        all_lemmposedsts.append(lemmposedws)
    
    all_fd = nltk.FreqDist(lemmposedwords)
    
    return all_posedsts, all_lemmposedsts, all_fd


### Words Importance metrics

In [5]:
############################
## wordimportance_var1: original
############################
def wordimportance_var1(all_lemmposedsts, all_fd):
    '''
    description:
    
    This metric was originally called "enlargedopacity"; renamed as wordimportance_var1.
    
    "enlargedopacity" is just a variation of the TF-IDF metric to measure word importance; the naming is just arbitrary and related to its use in this specific project.
    
    The importance of a word in this case is based on the combined effect of two metrics:
    --- opacity: 1 - total count of word against total count of the most frequent word (1 - log(f_w)/log(f_maxw)):
        this range between [0,1); the metric will penalize those words that are too frequent in the corpus, giving it a small value (close to 0)
    --- sizing: 1 - (max count of a word in a text) / (total count of the word in all texts):
        this range between [0,1); this metric is mostly a dispersion metric: the smaller the value, the larger the likelihood
        that the word concentrates in only one text; it also penalises very rare words

    opacity is about a complement of a TF variation (a standard TF is corpus' term frequency).
    
    sizing is about a complement of a IDF variation (a standard IDF is log( (Total Documents) / (Documents containing the term) )).
 
    While opacity will favour the less frequent words, sizing will adjust opacity so those that are too concentrated in very few texts (too rare) get penalized.
   
    Using sizing as dispersion metric, words that are more common between different text receive better ranking.

    "enlargedopacity" is the product of the frequency of the word against the most frequent word by its dispersion in the corpus, just like TF-IDF.
    Its calculation doesn't deviate much from other variations of the standard metric (https://en.wikipedia.org/wiki/Tf%E2%80%93idf).
      
    The idea is to highlight those words that are more shared in the corpus without being too frequent ones. Those are more like (shared) topic words.

    However:
    --- Be aware that the metric is not normalized by text's length: the effect of the frequent appearance of a word because the text is long is not considered
        This could affect mostly the dispersion metric.
    --- The metric favours words that are shared; for topic modelling it will excludes words that define topics specific to few texts
    --- The metrics assign value of (close to) 0 to penalised words: that makes those words to dissappear from the radar; it is like a hard L1 measure 
    --- In fact, it assigns value of 0 to less and very frequent words, suggesting they are the same when they are not
    
    input:
        1) tokenized list of texts
        2) freqDist of lemmatized words
    
    output: wordimportance_var1 
    '''

    maxdiv = math.log(sorted(all_fd.items(), key=lambda x: x[1], reverse=True)[0][1])
    opacity = dict([(k, 1-math.log(v)/maxdiv) for k,v in all_fd.items()])
    sizing_matrix = dict([(k, [0]*len(norm_posedsts)) for k in list(all_fd.keys())])
    data_out = []
    l = 0
    for i,lemmpos_t in enumerate(all_lemmposedsts):
        for k, lemmpos_w in enumerate(lemmpos_t):
            sizing_matrix[lemmpos_w][i] = sizing_matrix[lemmpos_w][i] + 1
       
    sizing = dict([(k, 1 - max(vector)/sum(vector)) for k, vector in sizing_matrix.items()])
    
    wordimportance_var1 = dict([(k, valsizing*opacity[k]) for k, valsizing in sizing.items()])

    return wordimportance

############################
## wordimportance_var2: modified to fit my heuristic for word ranking 
############################
def wordimportance_var2(all_lemmposedsts, all_fd):
    '''
    description:
    
    This metric tries to solve some of the issues that appeared in `wordimportance_var1` metric, in particular the values of zero.
    
    This is done by setting lower bounds when required.
    
    --- In the case of opacity, a non-zero lower bound is set by changing the equation to the following:
        ```
        if 1-math.log(v)/maxdiv == 0: 1-math.log(maxdiv-1)/maxdiv # 1-math.log(v)/maxdiv == 0 if v == maxdiv
        ```
    --- In the case of sizing, a redefinition of the metric force a non-zero lower bound as well as rebumpimg rare terms in documents:
        ```
        (sum(vector)-max(vector))/sum(vector)
        ```
    
    input:
        1) tokenized list of texts
        2) freqDist of lemmatized words
    
    output: wordimportance 
    '''

    maxdiv = math.log(sorted(all_fd.items(), key=lambda x: x[1], reverse=True)[0][1])
    opacity = dict([(k, 1-math.log(v)/maxdiv) if 1-math.log(v)/maxdiv != 0 else (k,1-math.log(maxdiv-1)/maxdiv) for k,v in all_fd.items()])
    sizing_matrix = dict([(k, [0]*len(all_lemmposedsts)) for k in list(all_fd.keys())])
    data_out = []
    l = 0
    for i,lemmpos_t in enumerate(all_lemmposedsts):
        for k, lemmpos_w in enumerate(lemmpos_t):
            sizing_matrix[lemmpos_w][i] = sizing_matrix[lemmpos_w][i] + 1
       
    sizing = dict([(k, (sum(vector)-max(vector))/sum(vector)) for k, vector in sizing_matrix.items()])
    
    wordimportance = dict([(k, valsizing*opacity[k]) for k, valsizing in sizing.items()])

    return wordimportance


############################
## wordimportance_var3: wordimportance_var2 modified to fit allrecordsPreparation2
############################
def wordimportance_var3(lemmposrecs, lemmws_fd):
    '''
    description:
    
    This metric tries to solve some of the issues that appeared in `wordimportance_var1` metric, in particular the values of zero.
    
    This is done by setting lower bounds when required.
    
    --- In the case of opacity, a non-zero lower bound is set by changing the equation to the following:
        ```
        if 1-math.log(v)/maxdiv == 0: 1-math.log(maxdiv-1)/maxdiv # 1-math.log(v)/maxdiv == 0 if v == maxdiv
        ```
    --- In the case of sizing, a redefinition of the metric force a non-zero lower bound as well as rebumpimg rare terms in documents:
        ```
        (sum(vector)-max(vector))/sum(vector)
        ```
    
    input:
        1) tokenized list of texts
        2) freqDist of lemmatized words
    
    output: wordimportance 
    '''

    maxdiv = math.log(sorted(lemmws_fd.items(), key=lambda x: x[1], reverse=True)[0][1])
    opacity = dict([(k, 1-math.log(v)/maxdiv) if 1-math.log(v)/maxdiv != 0 else (k,1-math.log(maxdiv-1)/maxdiv) for k,v in lemmws_fd.items()])
    sizing_matrix = dict([(k, [0]*len(lemmposrecs)) for k in list(lemmws_fd.keys())])

    ## Count lemmatized words/characters per text  
    for i,lemmpos_t in enumerate(lemmposrecs):
        for k, lemmpos_TUPLE in enumerate(lemmpos_t[3]):
            ## Use lemmatized word
            lemmw = lemmpos_TUPLE[2]
            sizing_matrix[lemmw][i] = sizing_matrix[lemmw][i] + 1
       
    ## Normalization
    normalization = dict([(k, (sum(vector)-max(vector))/sum(vector)) for k, vector in sizing_matrix.items()])
    
    wordimportance = dict([(k, valnorm*opacity[k]) for k, valnorm in normalization.items()])

    return wordimportance

# UNDER TEST

## Topic Modeling

### Stopwords and Punctuation removal

A very simple one

In [6]:
def cleaningtext(st, STOPWORDS = nltk.corpus.stopwords.words('english')):
    treated_st = []
    countwds = len(st)
    for w in st:
        if re.match(r'\w+', w) and not re.match(r'\d+?', w) and w not in ('merely', 'truly') and w not in STOPWORDS:
            if w == 'freecodecamp':
                w = 'fcc'
            treated_st.append(w)
    return treated_st, countwds



### Variations of Gensim Models for LSI and LDA

In [7]:
############################
## gensim_models1: substantial modification of original to embed data transformations within this function
############################

def gensim_models1(all_lemmposedsts, all_fd = {}, wordimportance = {}):
    NUM_TOPICS = 15
    STOPWORDS = nltk.corpus.stopwords.words('english') 
    redo_corpus_by_sts = []
    for lemmpos_t in all_lemmposedsts:
        lemmpos_sts = lemmpos_t.split('.')
        for lemmpos_st in lemmpos_sts:
            redo_corpus_by_sts.append(cleaningtext(lemmpos_st)[0])
    
    def basedonBOW(redo_corpus_by_sts):
        dictionary = gensim.corpora.Dictionary(redo_corpus_by_sts)
        corpus = [dictionary.doc2bow(text) for text in redo_corpus_by_sts]
    return corpus, dictionary
    
    def basedonTFIDF(corpus):
        return gensim.models.TfidfModel(corpus)
    
    def basedonOTHER(redo_corpus_by_sts, dictionary, wordimportance):
        def metriccalc1(w):
            return 1.0+2.0**float(wordimportance[w])
        
        corpus = []
        for sts in redo_corpus_by_sts:
            st = []
            for w in sts:
                st.append((dictionary.token2id[w], metriccalc1(w)))
            corpus.append(st)
        return corpus
        
    corpus, dictionary = basedonBOW(redo_corpus_by_sts)
    if wordimportance == {'tfidf':True}:
        tfidf = basedonTFIDF(corpus)
        corpus = tfidf[corpus]
    if wordimportance != {} and wordimportance != {'tfidf':True}:
        corpus = basedonOTHER(redo_corpus_by_sts, dictionary, wordimportance)
    
    
    lda_model = gensim.models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
    lsi_model = gensim.models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

   
    return lda_model, lsi_model

############################
## gensim_models2: modification of gensim_model1 to fit allrecordsPreparation2
############################

def gensim_models2(lemmposrecs, NUM_TOPICS = 15, lemmws_fd = {}, wordimportance = {}, nltk = nltk, gensim = gensim):
    
    def cleanedsts_from_lemmpostxts(lemmposrecs, STOPWORDS = nltk.corpus.stopwords.words('english')):
        redo_corpus_by_sts = []
        for lemmpos_r in lemmposrecs:
            lemmpos_t = lemmpos_r[3]
            lemm_sts = ''
            for lemmpos_TUPLE in lemmpos_t:
                w = lemmpos_TUPLE[2]
                lemm_sts = lemm_sts + w + ' '
            sts = lemm_sts.split('.')
            for st in sts:
                redo_corpus_by_sts.append(cleaningtext(st.split())[0])
        
        #print(len(redo_corpus_by_sts), redo_corpus_by_sts[-1])
        return redo_corpus_by_sts
        
    def basedonBOW(redo_corpus_by_sts):
        dictionary = gensim.corpora.Dictionary(redo_corpus_by_sts) #[token for st in redo_corpus_by_sts for token in st]
        corpus = [dictionary.doc2bow(text) for text in redo_corpus_by_sts]
        return corpus, dictionary
    
    def basedonTFIDF(corpus):
        return gensim.models.TfidfModel(corpus)
    
    def basedonOTHER(redo_corpus_by_sts, dictionary, wordimportance):
        
        def metriccalc(w):
            return 1.0+2.0**float(wordimportance[w])

        corpus = []
        for sts in redo_corpus_by_sts:
            st = []
            for w in sts:
                st.append((dictionary.token2id[w], metriccalc(w)))
            corpus.append(st)
        return corpus
        
    
    redo_corpus_by_sts  = cleanedsts_from_lemmpostxts(lemmposrecs)
    
    corpus, dictionary = basedonBOW(redo_corpus_by_sts)
    if wordimportance == {'tfidf':True}:
        tfidf = basedonTFIDF(corpus)
        corpus = tfidf[corpus]
    if wordimportance != {} and wordimportance != {'tfidf':True}:
        corpus = basedonOTHER(redo_corpus_by_sts, dictionary, wordimportance)
    
    
    lda_model = gensim.models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
    lsi_model = gensim.models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

   
    return lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary


### A more Raw LDA implementation

In [21]:
############################
## raw_lda_frankjupyter1: original
############################
def raw_lda_frankjupyter1(all_lemmposedsts, wordimportance):
    '''
    description: modified model based on https://www.frankcleary.com/svd/ for a more raw construction of a lda
    '''
    
    def metriccalc(st, normalizer, wordimportance):
        '''
        description:
        text normalization based on ALL characters in the sentence; why? Example: if two writers wrote 20 words, 2 of them very important, but one of them wrote half of characters stopwords, those 2 words wouldnt be penalized accordingly for this writer: the other wrote more important content
        '''       
        likedict = collections.defaultdict(float)
        textbow = collections.Counter(st)
        for w in st:
            #likedict[w] = math.pow(0.1+float(wordimportance[w]),textbow[w]/normalizer) #a sort of idf-normalization based on number of words in the text: the more the words in a text, the more important
            #likedict[w] = float(wordimportance[w])*textbow[w] #good but ignore those words with worimportance too low or 0 but that are frequent in text
            likedict[w] = 1.0+2.0**float(wordimportance[w]) #because it is not normilized this indicator would simply say that if it has the word at least once is already on topic
        return likedict
            
            
    STOPWORDS = nltk.corpus.stopwords.words('english')
    #redo_corpus_by_sts = []
    words_df = pandas.DataFrame()
    textreference = {}
    for textindex, norm_t in enumerate(all_lemmposedsts):
        print('norm_t', len(norm_t))
        redo_corpus_by_sts = []
        lensts = []
        norm_sts = norm_t.split('.')
        for stindex, lemmpos_st in enumerate(sts):
            treated_st, lensts = cleaningtext(lemmpos_st)        
            print('treated_st', lensts)
            if len(treated_st) > 3:
                likedict = metriccalc(treated_st, lensts[stindex], wordimportance)
                st_df = pandas.DataFrame.from_dict(likedict, orient='index')
                textindexing = str(textindex)+'_'+str(stindex)
                st_df.columns = [textindexing]
                textreference[textindexing] = {}
                textreference[textindexing]['treated_st'] = treated_st
                #st_df.columns = [str(count)]
                words_df = words_df.join(st_df, how='outer', )
    
    words_df = words_df.fillna(0)
    print("Number of unique words: %s" % len(words_df))
    print(words_df.sort(columns=words_df.columns[0], ascending=False).head(10))
    
    return words_df, textreference

############################
## raw_lda_frankjupyter2: modified to fit allrecordsPreparation2
############################
def raw_lda_frankjupyter2(lemmposrecs, wordimportance, metmodel=2, STOPWORDS=nltk.corpus.stopwords.words('english')):
    '''
    description: modified model based on https://www.frankcleary.com/svd/ for a more raw construction of a lda
    '''
    
   
    def metriccalc(st, normalizer, wordimportance):
        '''
        description:
        text normalization based on ALL characters in the sentence; why? Example: if two writers wrote 20 words, 2 of them very important, but one of them wrote half of characters stopwords, those 2 words wouldnt be penalized accordingly for this writer: the other wrote more important content
        '''       
        metfuncs = [
            lambda w: math.pow(0.1+float(wordimportance[w]),textbow[w]/normalizer) if w in list(wordimportance.keys()) else 0.0, #a sort of idf-normalization based on number of words in the text: the more the words in a text, the more important
            lambda w: float(wordimportance[w])*textbow[w], #good but ignore those words with worimportance too low or 0 but that are frequent in text
            lambda w: 1.0+2.0**float(wordimportance[w]) if w in list(wordimportance.keys()) else 0.0, #<- probably the best one; because it is not normilized this indicator would simply say that if it has the word at least once is already on topic
            lambda w: float(wordimportance[w]) + textbow[w]/normalizer,
            lambda w: (1.0+textbow[w]/normalizer)*float(wordimportance[w]), #<- apparently the second best with the adhoc word ranking
            lambda w: normalizer/textbow[w]*wordimportance[w]/sum(list(wordimportance.values())) #tfidf
        ]
        likedict = collections.defaultdict(float)
        textbow = collections.Counter(st)
        for w in st:
            likedict[w] = metfuncs[metmodel](w)
        return likedict

    #redo_corpus_by_sts = []
    words_df = pandas.DataFrame()
    textreference = {}
    
    
    for textindex, lemmpos_r in enumerate(lemmposrecs):
        lemmpos_t = lemmpos_r[3]
        #print('lemmpos_t', len(lemmpos_t))
        lemm_sts = ''
        for lemmpos_TUPLE in lemmpos_t:
            w = lemmpos_TUPLE[2]
            lemm_sts = lemm_sts + w + ' '
        sts = lemm_sts.split('.')
        for stindex, lemmpos_st in enumerate(sts):
            treated_st, lensts = cleaningtext(lemmpos_st.split(), STOPWORDS=STOPWORDS)        
            #print('treated_st', lensts)
            if len(treated_st) > 3:
                likedict = metriccalc(treated_st, lensts, wordimportance)
                st_df = pandas.DataFrame.from_dict(likedict, orient='index')
                textindexing = str(textindex)+'_'+str(stindex)
                st_df.columns = [textindexing]
                textreference[textindexing] = {}
                textreference[textindexing]['treated_st'] = treated_st
                #st_df.columns = [str(count)]
                words_df = words_df.join(st_df, how='outer', )
    
    words_df = words_df.fillna(0)
    print("Number of unique words: %s" % len(words_df))
    print(words_df.head(10))
    #print(words_df.sort(columns=words_df.columns[0], ascending=False).head(10))
    
    return words_df, textreference

In [9]:
def dist(col1, col2, sigma):
    """
    Return the norm of (col1 - col2), where the differences in 
    each dimension are wighted by the values in sigma.
    import scipy
    numpy.where(words_df.iloc[:,0] != 0)
    scipy.spatial.distance.cosine(words_df.iloc[:,0],words_df.iloc[:,1])
    words_df.index[numpy.where(words_df.iloc[:,0] != 0)].values, words_df.index[numpy.where(words_df.iloc[:,1] != 0)].values
    sklearn.metrics.pairwise.cosine_similarity(words_df.loc[:,'0_0'],words_df.loc[:,'3_12'])
    1-scipy.spatial.distance.cosine(words_df.loc[:,'0_0'],words_df.loc[:,'3_12'])
    """
    #return numpy.linalg.norm(numpy.array(col1 - col2) * sigma)
    #return 1 - scipy.spatial.distance.cosine(col1,col2) #always saying they are different; also http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
    #return (1 - scipy.spatial.distance.cosine(col1,col2))*numpy.linalg.norm(numpy.array(col1 - col2) * sigma)
    #return (1 - scipy.spatial.distance.cosine(col1,col2*sigma))
    return (scipy.spatial.distance.cosine(col1,col2*sigma))

def similarityanalysis(words_df):
    
    U, sigma, V = numpy.linalg.svd(words_df)
    v_df = pandas.DataFrame(V, columns=words_df.columns)
    v_df.apply(lambda x: numpy.round(x, decimals=2))
     
    dist_df = pandas.DataFrame(index=v_df.columns, columns=v_df.columns)
    for cname in v_df.columns:
        dist_df[cname] = v_df.apply(lambda x: dist(v_df[cname].values, x.values, sigma))
    
    return dist_df, U, sigma, V, v_df

## Running LDA and LSI models

In [10]:
with open('../data/jobproject_forum.json','r') as message:
    otp = json.load(message)
print(len(otp))
allrecords = [{ "user": k, "data": otp[k] }  for k in otp]
print(len(allrecords))

90
90


In [11]:
lemmposrecs, lemmws_fd = allrecordsPreparation2(allrecords)

in allrecordsPreparation (len(allrecords)):: 90


/home/ec/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


pos KeyErrors in IN
pos KeyErrors i JJ
pos KeyErrors a DT
pos KeyErrors but CC
pos KeyErrors on IN
pos KeyErrors around IN
pos KeyErrors new JJ
pos KeyErrors live JJ
pos KeyErrors it PRP
pos KeyErrors i JJ
pos KeyErrors any DT
pos KeyErrors though IN
pos KeyErrors all DT
pos KeyErrors a DT
pos KeyErrors simple JJ
pos KeyErrors much JJ
pos KeyErrors it PRP
pos KeyErrors a DT
pos KeyErrors no DT
pos KeyErrors no DT
pos KeyErrors real JJ
IndexErrors (not found) i VB
pos KeyErrors as IN
pos KeyErrors possible JJ
pos KeyErrors a DT
pos KeyErrors few JJ
pos KeyErrors a DT
pos KeyErrors polish JJ
pos KeyErrors a DT
pos KeyErrors after IN
pos KeyErrors first JJ
pos KeyErrors in IN
pos KeyErrors it PRP
pos KeyErrors much JJ
pos KeyErrors dumb JJ
pos KeyErrors luck LS
pos KeyErrors a DT
pos KeyErrors junior JJ
IndexErrors (not found) asap NN
IndexErrors (not found) i VBP
pos KeyErrors a DT
pos KeyErrors modest JJ
pos KeyErrors i JJ
pos KeyErrors several JJ
pos KeyErrors a DT
pos KeyErrors a DT
p

In [12]:
wordimportance = wordimportance_var3(lemmposrecs, lemmws_fd)

In [25]:
words_df, textreference = raw_lda_frankjupyter2(lemmposrecs, wordimportance, metmodel=0)

Number of unique words: 2569
             0_0  0_1  0_3  0_4  0_5  0_6  0_7  0_8  0_11  0_12  ...    \
ability      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
able         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
abroad       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
absolute     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
absolutely   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
absorb       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
academy      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
accelerator  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
accept       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
access       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     

                52_1  52_2  52_3  52_4  52_5  52_6  52_7  52_8  52_9  52_10  
ability      0.00000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  


In [ ]:
words_df.iloc[(numpy.where(words_df.iloc[:,0] != 0))]

In [ ]:
words_df.iloc[(numpy.where(words_df.iloc[:,0] != 0))]

In [26]:
dist_df, U, sigma, V, v_df = similarityanalysis(words_df)

In [ ]:
textreference['0_0']

In [29]:
counter = 0
for paper in dist_df.columns:
    sim_papers_df = dist_df.sort(columns=paper)[paper]
    #print(sim_papers_df)
    sim_papers = sim_papers_df.drop([paper]).index
    #print(sim_papers)
    print('Papers most similar to ' + paper + ':')
    print(paper + ':' + ' '.join(textreference[paper]['treated_st']))
    print(', '.join(sim_papers[:10]))
    for ind in sim_papers[:10]:
        print(ind + ':' + ' '.join(textreference[ind]['treated_st']), sim_papers_df[ind])
    print( '\n')
    counter += 1
    if counter > 5:
        break

Papers most similar to 0_0:
0_0:hi god i’ve wait post something section since i’ve join fcc join community year ago start work fcc challenge around october beta new curriculum go live i’ve switch didn’t even get certificate yet even though finish beta challenge
2_15, 29_26, 44_0, 7_9, 36_8, 27_10, 31_2, 39_22, 50_0, 45_0
2_15:finish front end certificate end october 0.811086271966
29_26:story year since join fcc 0.832982353605
44_0:hi everyone join fcc two month ago dream post 0.843989316358
7_9:codecs start work fcc curriculum 0.865512635701
36_8:finish fcc certificate march 0.874024409891
27_10:join fcc create account since already finish course year prior know fcc 0.882787098518
31_2:start fcc year ago complete frontend certificate along udemy course 0.887178936799
39_22:structured curriculum nice community important 0.891480391487
50_0:hi camper first thanks freecodecamps amazing community around project 0.893916009399
45_0:venezuela start code year ago 0.895931024317


Papers most

/home/ec/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [30]:
words_df, textreference = raw_lda_frankjupyter2(lemmposrecs, wordimportance, metmodel=2)

Number of unique words: 2569
             0_0  0_1  0_3  0_4  0_5  0_6  0_7  0_8  0_11  0_12  ...    \
ability      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
able         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
abroad       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
absolute     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
absolutely   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
absorb       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
academy      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
accelerator  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
accept       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
access       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     

                52_1  52_2  52_3  52_4  52_5  52_6  52_7  52_8  52_9  52_10  
ability      0.00000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  


In [31]:
dist_df, U, sigma, V, v_df = similarityanalysis(words_df)

In [32]:
counter = 0
for paper in dist_df.columns:
    sim_papers_df = dist_df.sort(columns=paper)[paper]
    #print(sim_papers_df)
    sim_papers = sim_papers_df.drop([paper]).index
    #print(sim_papers)
    print('Papers most similar to ' + paper + ':')
    print(paper + ':' + ' '.join(textreference[paper]['treated_st']))
    print(', '.join(sim_papers[:10]))
    for ind in sim_papers[:10]:
        print(ind + ':' + ' '.join(textreference[ind]['treated_st']), sim_papers_df[ind])
    print( '\n')
    counter += 1
    if counter > 5:
        break

Papers most similar to 0_0:
0_0:hi god i’ve wait post something section since i’ve join fcc join community year ago start work fcc challenge around october beta new curriculum go live i’ve switch didn’t even get certificate yet even though finish beta challenge
2_15, 29_26, 44_0, 7_9, 36_8, 27_10, 31_2, 39_22, 50_0, 45_0
2_15:finish front end certificate end october 0.808138777096
29_26:story year since join fcc 0.835212939007
44_0:hi everyone join fcc two month ago dream post 0.842545077788
7_9:codecs start work fcc curriculum 0.86800977926
36_8:finish fcc certificate march 0.876360774279
27_10:join fcc create account since already finish course year prior know fcc 0.886641327604
31_2:start fcc year ago complete frontend certificate along udemy course 0.890158902846
39_22:structured curriculum nice community important 0.891954081808
50_0:hi camper first thanks freecodecamps amazing community around project 0.892326529223
45_0:venezuela start code year ago 0.898436800568


Papers most 

/home/ec/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [ ]:
words_df, textreference = raw_lda_frankjupyter2(lemmposrecs, wordimportance, metmodel=3)

In [ ]:
dist_df, U, sigma, V, v_df = similarityanalysis(words_df)

In [ ]:
counter = 0
for paper in dist_df.columns:
    sim_papers_df = dist_df.sort(columns=paper)[paper]
    #print(sim_papers_df)
    sim_papers = sim_papers_df.drop([paper]).index
    #print(sim_papers)
    print('Papers most similar to ' + paper + ':')
    print(paper + ':' + ' '.join(textreference[paper]['treated_st']))
    print(', '.join(sim_papers[:10]))
    for ind in sim_papers[:10]:
        print(ind + ':' + ' '.join(textreference[ind]['treated_st']), sim_papers_df[ind])
    print( '\n')
    counter += 1
    if counter > 5:
        break

In [ ]:
words_df, textreference = raw_lda_frankjupyter2(lemmposrecs, wordimportance, metmodel=4)

In [ ]:
dist_df, U, sigma, V, v_df = similarityanalysis(words_df)

In [ ]:
counter = 0
for paper in dist_df.columns:
    sim_papers_df = dist_df.sort(columns=paper)[paper]
    #print(sim_papers_df)
    sim_papers = sim_papers_df.drop([paper]).index
    #print(sim_papers)
    print('Papers most similar to ' + paper + ':')
    print(paper + ':' + ' '.join(textreference[paper]['treated_st']))
    print(', '.join(sim_papers[:10]))
    for ind in sim_papers[:10]:
        print(ind + ':' + ' '.join(textreference[ind]['treated_st']), sim_papers_df[ind])
    print( '\n')
    counter += 1
    if counter > 5:
        break

In [17]:
words_df, textreference = raw_lda_frankjupyter2(lemmposrecs, lemmws_fd, metmodel=5)

Number of unique words: 2569
             0_0  0_1  0_3  0_4  0_5  0_6  0_7  0_8  0_11  0_12  ...    \
ability      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
able         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
abroad       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
absolute     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
absolutely   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
absorb       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
academy      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
accelerator  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
accept       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     
access       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  ...     

                 52_1  52_2  52_3  52_4  52_5  52_6  52_7  52_8  52_9  52_10  
ability      0.000000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0 

In [18]:
dist_df, U, sigma, V, v_df = similarityanalysis(words_df)

In [19]:
counter = 0
for paper in dist_df.columns:
    sim_papers_df = dist_df.sort(columns=paper)[paper]
    #print(sim_papers_df)
    sim_papers = sim_papers_df.drop([paper]).index
    #print(sim_papers)
    print('Papers most similar to ' + paper + ':')
    print(paper + ':' + ' '.join(textreference[paper]['treated_st']))
    print(', '.join(sim_papers[:10]))
    for ind in sim_papers[:10]:
        print(ind + ':' + ' '.join(textreference[ind]['treated_st']), sim_papers_df[ind])
    print( '\n')
    counter += 1
    if counter > 5:
        break

Papers most similar to 0_0:
0_0:hi god i’ve wait post something section since i’ve join fcc join community year ago start work fcc challenge around october beta new curriculum go live i’ve switch didn’t even get certificate yet even though finish beta challenge
22_11, 27_11, 27_5, 47_10, 42_4, 31_2, 2_0, 43_5, 27_4, 7_9
22_11:udacity acknowledge hole since pledge updated curriculum 0.716493104609
27_11:year ago desperation finish course quality assurance start apply qa analyst success 0.731003448269
27_5:stay home raise family year ago start comeback profession 0.739722498552
47_10:year ago wasn’t able properly type finger 0.756303137735
42_4:n't finish college since need stop start work meet family need rich 0.758207344387
31_2:start fcc year ago complete frontend certificate along udemy course 0.759659976998
2_0:hello camper start fcc little less year ago 0.761746829959
43_5:learn start fcc couple year ago go couple spurt i’d systematically progress month life get way i’d stall nothi

/home/ec/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [ ]:
import scipy
numpy.where(words_df.iloc[:,0] != 0)
scipy.spatial.distance.cosine(words_df.iloc[:,0],words_df.iloc[:,1])
words_df.index[numpy.where(words_df.iloc[:,0] != 0)].values, words_df.index[numpy.where(words_df.iloc[:,1] != 0)].values
sklearn.metrics.pairwise.cosine_similarity(words_df.loc[:,'0_0'],words_df.loc[:,'3_12'])
1-scipy.spatial.distance.cosine(words_df.loc[:,'0_0'],words_df.loc[:,'3_12'])

In [ ]:
lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, wordimportance = {'tfidf':True})

In [ ]:
lda_model.print_topics()

In [ ]:
lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, wordimportance = wordimportance)

In [ ]:
lda_model.print_topics()

In [ ]:
lsi_model.print_topics()

## Keyphrase Extraction and Graph Analysis

In [ ]:
#TODO